In [2]:
import pandas as pd
import numpy as np
import os
import pyodbc

In [29]:
year = 2022
semana = 1
path = 'Medios/Contratos/Week ' + str(semana) + ' ' + str(year)
path = 'Medios/Contratos/Q1 2022'
file_names = os.listdir(path)

In [56]:
dict_contracts = {169941:'UDN TIER I',
                169942:'UNV TIER I',
                169943:'UNV TIER II',
                169944:'UNV TIER III',
                170780:'UNV SUB A I',
                170781:'UNV SUB A II',
                169953:'GALA TIER I',
                169940:'UMA TIER I',
                169949:'UMA TIER II',
                169954:'UMA TIER III',
                170785:'UMA SUB A I',
                170786:'UMA SUB A II',
                176732:'UMA UVN UD',
                177261:'GALA UVN UD',
                176929: 'UMA TIER I',
                176931: 'UMA TIER III',
                176934: 'GALA TIER I',
                176982: 'UVN TIER I',
                176983: 'UVN TIER II',
                176984: 'TUDN TIER I',
                176985: 'UVN TIER III',
                177011: 'UMA TIER II',
                177423: 'UVN SUB TIER I',
                177424: 'UVN SUB TIER II',
                177426: 'UMA SUB A TIER I',
                177428: 'UMA SUB TIER II'}

In [25]:
file_names

['176929 UMA Base Tier I.xls',
 '176931 UMA Base Tier III.xls',
 '176934 GALA Base Tier I.xls',
 '176982 UVN Base Tier I.xls',
 '176983 UVN Base Tier II.xls',
 '176984 TUDN Base Tier I.xls',
 '176985 UVN Base Tier III.xls',
 '177011 UMA Base Tier II.xls',
 '177423 UVN SubNet Tier I.xls',
 '177424 UVN SubNet Tier II.xls',
 '177426 UMA Subnet A Tier I.xls',
 '177428 UMA SubNet Tier II.xls']

## Number of spots

In [57]:
ids_cont = []
for file in file_names:
    ids_cont.append(int(file.split(' ')[0]))

In [58]:
names = ['TUDN','Univision Simulcasts','Univision Network','UniMAS','GALAVISION']
dayparts = ['EARLY MORNING','DAYTIME','FRINGE','PRIME','LATE NIGHT','WKEND DAYTIME','SPORTS','NEWS','SPECIALS']

week = '12/27'
df_all = pd.DataFrame()

for file in file_names:
    contract_id = int(file.split(' ')[0])
    df = pd.read_excel(path + '/' + file,header=13)
    df = df.loc[1:,:].copy() #Remove planes row
    df['Daypart / Selling Title'] = df['Daypart / Selling Title'].fillna(0)

    last_row = df[df['Daypart / Selling Title'].astype(str).str.contains('Equivalized')].index[0]
    df = df.loc[:last_row-1,:].copy()
    df = df[df['Daypart / Selling Title'] != 0]
    df.reset_index(inplace=True,drop=True)

    df['Contract'] = df['Daypart / Selling Title'].apply(lambda x: x if x in names else np.NaN)
    df['Contract'] = df['Contract'].fillna(method='ffill')
    df['Daypart'] = df['Daypart / Selling Title'].apply(lambda x: x if x in dayparts else np.NaN)
    df['Daypart'] = df['Daypart'].fillna(method='ffill')
    df['Contract_id'] = contract_id
    df['Week'] = str(semana) + '-' + str(year)

    df = df[['Week','Contract','Contract_id','Daypart','Length',week]].copy()
    df['Spots'] = df[week].fillna(0)
    df.drop(week,axis=1,inplace=True)
    df.dropna(subset=['Length'],inplace=True)
    
    df_all = pd.concat([df_all,df])
    
df_all.reset_index(drop=True,inplace=True)
df_all['Contract_id'] = df_all['Contract_id'].map(dict_contracts)
df_all = df_all[df_all['Spots'] != 0].copy()

In [59]:
df_all.isnull().sum()

Week           0
Contract       0
Contract_id    0
Daypart        0
Length         0
Spots          0
dtype: int64

In [60]:
semana = 1
df_all.to_csv('Outputs/Flowcharts Review/Spots Detail/Spots Detail Week ' + str(semana) + '_'+ str(year) +'.csv',index=False)

## Impresions

In [61]:
names = ['TUDN','Univision Simulcasts','Univision Network','UniMAS','GALAVISION']
dayparts = ['EARLY MORNING','DAYTIME','FRINGE','PRIME','LATE NIGHT','WKEND DAYTIME','SPORTS','NEWS','SPECIALS']

df_all = pd.DataFrame()

first_week = '12/27' #Input first week on flowchart
last_week = '03/21'  #Input last week on flowchart

for file in file_names:
    
    #Read file
    contract_id = int(file.split(' ')[0])
    df = pd.read_excel(path + '/' + file,header=13)
    #Remove planes row
    df = df.loc[1:,:].copy() 
    columns = np.array(df.columns)
    colfin = np.where(columns == last_week)[0][0] 
    #filter columns 0 to last_week
    df = df.iloc[:,:colfin+1] 
    df['Daypart / Selling Title'] = df['Daypart / Selling Title'].fillna(0)
    
    #Remove bottom useless rows 
    last_row = df[df['Daypart / Selling Title'].astype(str).str.contains('Equivalized')].index[0] 
    df = df.loc[:last_row-1,:].copy()
    #Remove null rows on first column
    df = df[df['Daypart / Selling Title'] != 0]
    df.reset_index(inplace=True,drop=True)
    
    #Copy the contract into another column and forward fill
    df['Contract'] = df['Daypart / Selling Title'].apply(lambda x: x if x in names else np.NaN)
    df['Contract'] = df['Contract'].fillna(method='ffill')
    #Copy the daypart into another column and forward fill
    df['Daypart'] = df['Daypart / Selling Title'].apply(lambda x: x if x in dayparts else np.NaN)
    df['Daypart'] = df['Daypart'].fillna(method='ffill')
    df['Contract_id'] = contract_id
    
    #Rename column Imps to (000) (NEEDED ONLY FOR TUDN)
    df.rename({'Imps':'(000)'},axis=1,inplace=True)
    columns = df.columns
    df.columns = [i.strip() for i in columns]
    columns = np.array(df.columns)
    
    #Find the impresions column
    col = np.where(columns == '(000)')[0][0]
    #Find the first week column
    col2 = np.where(columns == first_week)[0][0]
    #Rename column (000) to Imps
    df.rename({df.columns[col]:'Imps'},axis=1,inplace=True)
    #Multiply the Imps by 1000 (NOT NEEDED FOR TUDN)
    if df['Contract_id'].unique()[0] != 169941:
        df['Imps'] = df['Imps']*1000
    #Sum the number of spots in the quarter line by line
    df['Total Q Spots'] = df.iloc[:,col2:col2+13].sum(axis=1)
    
    #Remove useless columns
    df = df[['Contract','Contract_id','Daypart','Length','Imps','Total Q Spots']].copy()
    #Filter 30 sec spots only
    df = df[df['Length']==':30']
    
    df_all = pd.concat([df_all,df])
    print(file)
    
df_all.reset_index(drop=True,inplace=True)
df_all['Contract_id'] = df_all['Contract_id'].map(dict_contracts)

176929 UMA Base Tier I.xls
176931 UMA Base Tier III.xls
176934 GALA Base Tier I.xls
176982 UVN Base Tier I.xls
176983 UVN Base Tier II.xls
176984 TUDN Base Tier I.xls
176985 UVN Base Tier III.xls
177011 UMA Base Tier II.xls
177423 UVN SubNet Tier I.xls
177424 UVN SubNet Tier II.xls
177426 UMA Subnet A Tier I.xls
177428 UMA SubNet Tier II.xls


In [62]:
df_all.shape

(142, 6)

In [63]:
df_all.isnull().sum()

Contract         0
Contract_id      0
Daypart          0
Length           0
Imps             0
Total Q Spots    0
dtype: int64

In [64]:
df_all.to_csv('Outputs/Flowcharts Review/Imps/ImpsQ12022.csv',index=False)

In [65]:
pivot = pd.pivot_table(df_all,index=['Contract','Contract_id','Daypart'],values=['Imps']).reset_index()
pivot['Imps'] = pivot['Imps'].apply(lambda x: round(x,0))
pivot

,Contract,Contract_id,Daypart,Imps
0,GALAVISION,GALA TIER I,DAYTIME,32000.0
1,GALAVISION,GALA TIER I,EARLY MORNING,13000.0
2,GALAVISION,GALA TIER I,FRINGE,36000.0
3,GALAVISION,GALA TIER I,LATE NIGHT,36000.0
4,GALAVISION,GALA TIER I,PRIME,81000.0
...,...,...,...,...
68,Univision Simulcasts,UVN TIER I,EARLY MORNING,227000.0
69,Univision Simulcasts,UVN TIER II,DAYTIME,227000.0
70,Univision Simulcasts,UVN TIER II,EARLY MORNING,227000.0
71,Univision Simulcasts,UVN TIER III,DAYTIME,227000.0


In [66]:
df.shape

(7, 6)

In [67]:
df_all.shape

(142, 6)

In [45]:
df_all

,Contract,Contract_id,Daypart,Length,Imps,Total Q Spots
0,UniMAS,UMA TIER I,DAYTIME,:30,107000.0,1.0
1,UniMAS,UMA TIER I,DAYTIME,:30,107000.0,1.0
2,UniMAS,UMA TIER I,DAYTIME,:30,56000.0,1.0
3,UniMAS,UMA TIER I,DAYTIME,:30,56000.0,14.0
4,UniMAS,UMA TIER I,DAYTIME,:30,77000.0,70.0
...,...,...,...,...,...,...
137,UniMAS,UMA SUB TIER II,DAYTIME,:30,48000.0,94.0
138,UniMAS,UMA SUB TIER II,FRINGE,:30,46000.0,44.0
139,UniMAS,UMA SUB TIER II,PRIME,:30,138000.0,52.0
140,UniMAS,UMA SUB TIER II,LATE NIGHT,:30,19000.0,72.0
